In [1]:
%matplotlib inline

import numpy as np
import tensorflow as tf
import time
import matplotlib.pyplot as plt
import pprint
pp = pprint.PrettyPrinter(indent=4)
from sklearn.model_selection import train_test_split

In [38]:
outputdir = 'D:/MyWorx/Courses/PAF Bootcamp/Challenges/Images/output'
imagesdata = np.load(outputdir+'/'+'images.npy')

print('number of samples: ', len(imagesdata))
print('Image Shape:',imagesdata[:,0][0].shape)
print('Labels: ', np.unique(imagesdata[:,1]))

number of samples:  2112
Image Shape: (128, 128, 3)
Labels:  ['axes' 'boots' 'carabiners' 'crampons' 'gloves' 'hardshell_jackets'
 'harnesses' 'helmets' 'insulated_jackets' 'pulleys' 'rope' 'tents']


In [46]:
# Parameter definitions
batch_size = 100
learning_rate = 0.005
max_steps = 1000

In [49]:
Images = imagesdata[:,0]
Labels = imagesdata[:,1]
classes = np.unique(imagesdata[:,1])

reshapedImages = []
for img in Images:
    img = np.reshape(img,49152)
    reshapedImages.append(img)
    
Images = np.array(reshapedImages)
print(Images.shape)
for x in range(12):
    Labels[Labels == classes[x]] = x

images_train, images_test,labels_train,labels_test = train_test_split(Images,Labels,test_size=0.2
                                                                     ,shuffle=True,random_state=42)

print(images_train.shape)
print(labels_train.shape)

(2112, 49152)
(1689, 49152)
(1689,)


In [50]:
#Data Points PlaceHolders (128x128x3)
images_placeholder = tf.placeholder(tf.float32, shape=[None, 49152])
labels_placeholder = tf.placeholder(tf.int64, shape=[None])

# Define variables (these are the values we want to optimize)

weights = tf.Variable(tf.zeros([49152, 12]))
biases = tf.Variable(tf.zeros([12]))

### Building the Model

In [51]:
# Define loss function
logits = tf.matmul(images_placeholder, weights) + biases
loss=tf.reduce_mean(tf.nn.sparse_softmax_cross_entropy_with_logits(logits=logits,labels=labels_placeholder))

# Define training operation
train_step = tf.train.GradientDescentOptimizer(learning_rate).minimize(loss)

# Operation comparing prediction with true label

correct_prediction = tf.equal(tf.argmax(logits, 1), labels_placeholder)

# Operation calculating the accuracy of our predictions
accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))


In [52]:
# Run the TensorFlow graph
with tf.Session() as sess:
    # Initialize variables
    sess.run(tf.global_variables_initializer())
    # Repeat max_steps times
    for i in range(max_steps):
        indices = np.random.choice(images_train.shape[0], batch_size)
        images_batch = images_train[indices]
        labels_batch = labels_train[indices]
        
        if i % 100 == 0:
            train_accuracy = sess.run(accuracy, feed_dict={
                images_placeholder: images_batch, labels_placeholder: labels_batch})
            print('Step {:5d}: training accuracy {:g}'.format(i, train_accuracy))
        
        sess.run(train_step, feed_dict={images_placeholder: images_batch,
                                        labels_placeholder: labels_batch})
    test_accuracy = sess.run(accuracy, feed_dict={
        images_placeholder: images_test,
        labels_placeholder: labels_test})
    print('Test accuracy {:g}'.format(test_accuracy))
    

Step     0: training accuracy 0.03
Step   100: training accuracy 0.47
Step   200: training accuracy 0.75
Step   300: training accuracy 0.83
Step   400: training accuracy 0.8
Step   500: training accuracy 0.8
Step   600: training accuracy 0.87
Step   700: training accuracy 0.86
Step   800: training accuracy 0.88
Step   900: training accuracy 0.83
Test accuracy 0.775414


In [ ]:
import pickle

pkl_filename = outputdir+'/'+'taditionalImageClassification.pkl'
with open(pkl_filename, 'wb') as file: 
    savedmodel = pickle.dump(sgd_clf,file)